In [1]:
import torch
import pandas as pd
from torch import nn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [2]:
data=pd.read_csv("process_house.csv")
x=data.iloc[:,:-1]
y=data.iloc[:,-1:]


In [3]:
y

,price
0,13300000
1,12250000
2,12250000
3,12215000
4,11410000
...,...
540,1820000
541,1767150
542,1750000
543,1750000


In [4]:
import joblib
scalex=StandardScaler()
scaley=StandardScaler()

x=scalex.fit_transform(x)
y=scaley.fit_transform(y)

joblib.dump(scalex,"scale_x.pkl")
joblib.dump(scaley,"scale_y.pkl")

scaley.inverse_transform(y)

array([[13300000.],
       [12250000.],
       [12250000.],
       [12215000.],
       [11410000.],
       [10850000.],
       [10150000.],
       [10150000.],
       [ 9870000.],
       [ 9800000.],
       [ 9800000.],
       [ 9681000.],
       [ 9310000.],
       [ 9240000.],
       [ 9240000.],
       [ 9100000.],
       [ 9100000.],
       [ 8960000.],
       [ 8890000.],
       [ 8855000.],
       [ 8750000.],
       [ 8680000.],
       [ 8645000.],
       [ 8645000.],
       [ 8575000.],
       [ 8540000.],
       [ 8463000.],
       [ 8400000.],
       [ 8400000.],
       [ 8400000.],
       [ 8400000.],
       [ 8400000.],
       [ 8295000.],
       [ 8190000.],
       [ 8120000.],
       [ 8080940.],
       [ 8043000.],
       [ 7980000.],
       [ 7962500.],
       [ 7910000.],
       [ 7875000.],
       [ 7840000.],
       [ 7700000.],
       [ 7700000.],
       [ 7560000.],
       [ 7560000.],
       [ 7525000.],
       [ 7490000.],
       [ 7455000.],
       [ 7420000.],


In [39]:
x=torch.tensor(x,dtype=torch.float)
y=torch.tensor(y,dtype=torch.float)

In [40]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,train_size=0.8,random_state=42)


In [41]:
torch.manual_seed(42)
model=nn.Linear(10,1)
model.state_dict()

OrderedDict([('weight',
              tensor([[ 0.2418,  0.2625, -0.0741,  0.2905, -0.0693,  0.0638, -0.1540,  0.1857,
                        0.2788, -0.2320]])),
             ('bias', tensor([0.2749]))])

In [42]:
loss_function=nn.MSELoss()
optimizer=torch.optim.Adam(params=model.parameters(),lr=0.01,weight_decay=1e-4)

In [45]:
ypred=model(xtrain)
loss_function(ypred,ytrain)

tensor(0.2918, grad_fn=<MseLossBackward0>)

In [44]:
epochs=10000
for epoch in range(epochs):
    model.train()
    ypred=model(xtrain)
    lossss=loss_function(ypred,ytrain)
    optimizer.zero_grad()
    lossss.backward()
    optimizer.step()
    model.eval()
    with torch.no_grad():
     test_pred=model(xtest)
     test_loss=loss_function(test_pred,ytest)
     print(f"epoch=({epoch}) || loss=({lossss}) || test-loss=({test_loss}) || r2-score=({r2_score(ytest,model(xtest).detach().numpy())}) ")

epoch=(0) || loss=(0.8210524916648865) || test-loss=(1.063436508178711) || r2-score=(0.26528799533843994) 
epoch=(1) || loss=(0.7725274562835693) || test-loss=(1.0184365510940552) || r2-score=(0.29637783765792847) 
epoch=(2) || loss=(0.7273140549659729) || test-loss=(0.9763085842132568) || r2-score=(0.3254833221435547) 
epoch=(3) || loss=(0.6854479312896729) || test-loss=(0.9370937943458557) || r2-score=(0.35257619619369507) 
epoch=(4) || loss=(0.6469273567199707) || test-loss=(0.9008034467697144) || r2-score=(0.3776487708091736) 
epoch=(5) || loss=(0.6116967797279358) || test-loss=(0.8674044609069824) || r2-score=(0.40072357654571533) 
epoch=(6) || loss=(0.5796303153038025) || test-loss=(0.8368082046508789) || r2-score=(0.42186206579208374) 
epoch=(7) || loss=(0.5505267977714539) || test-loss=(0.8088706731796265) || r2-score=(0.44116365909576416) 
epoch=(8) || loss=(0.5241303443908691) || test-loss=(0.7834103107452393) || r2-score=(0.4587538242340088) 
epoch=(9) || loss=(0.50017237663

In [260]:
torch.save(model.state_dict(),"model.pth")

In [290]:
model=nn.Linear(10,1)
model.load_state_dict(torch.load("model.pth"))
model.eval

C:\Users\Avijit\AppData\Local\Temp\ipykernel_16348\2719771831.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model.pth"))


<bound method Module.eval of Linear(in_features=10, out_features=1, bias=True)>

In [293]:
val=scalex.transform([[ 0.4056, -0.4653,  1.4726, -0.5540, -0.0917,  0.9453,  0.0473,  1.4218,
        -0.9294,  2.6794]])
input=torch.tensor(val,dtype=torch.float)
with torch.no_grad():
 val1=model(input)
 print(scaley.inverse_transform(val1.detach().numpy()))

[[3129172.5]]


c:\Users\Avijit\anaconda3\envs\DL\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
